In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from numpy import mean
from numpy import std
import numpy as np
from matplotlib import pyplot
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import confusion_matrix
from scipy.io import loadmat
import numpy as np
import PIL
import cv2
import os
from sklearn.model_selection import train_test_split

In [3]:
x = np.load('/content/drive/MyDrive/heart_sound_results_eval_new/database_physionet_challenge_all/x.npy')
y = np.load('/content/drive/MyDrive/heart_sound_results_eval_new/database_physionet_challenge_all/y.npy')

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=1/8, random_state=1)

y_tr_one_hot  = np.zeros((np.array(y_train).shape[0],2))
for i in range(np.array(y_train).shape[0]):
  label = y_train[i]
  y_tr_one_hot[i][int(label)] = 1

y_te_one_hot  = np.zeros((np.array(y_test).shape[0],2))
for i in range(np.array(y_test).shape[0]):
  label = y_test[i]
  y_te_one_hot[i][int(label)] = 1

y_val_one_hot  = np.zeros((np.array(y_val).shape[0],2))
for i in range(np.array(y_val).shape[0]):
  label = y_val[i]
  y_val_one_hot[i][int(label)] = 1

In [14]:
model = tf.keras.applications.VGG19(include_top=False,input_shape=(128,128,3))
# mark loaded layers as not trainable
for layer in model.layers:
	layer.trainable = False
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
output = Dense(2, activation='softmax')(flat1)
model = Model(inputs=model.inputs, outputs=output)

optimizer = tf.keras.optimizers.Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(np.array(x_train), y_tr_one_hot, validation_data=(np.array(x_val), y_val_one_hot), epochs=10, batch_size=512,verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10
4/4 [==============================] - 491s 127s/step - loss: 5.3218 - accuracy: 0.6370 - val_loss: 4.8748 - val_accuracy: 0.8223
Epoch 2/10
4/4 [==============================] - 484s 126s/step - loss: 4.1881 - accuracy: 0.8003 - val_loss: 1.4202 - val_accuracy: 0.7491
Epoch 3/10
4/4 [==============================] - 482s 125s/step - loss: 2.5429 - accuracy: 0.5572 - val_loss: 1.1447 - val_accuracy: 0.6829
Epoch 4/10
4/4 [==============================] - 488s 125s/step - loss: 1.6855 - accuracy: 0.7748 - val_loss: 2.0419 - val_accuracy: 0.8258
Epoch 5/10
4/4 [==============================] - 486s 126s/step - loss: 1.5431 - accuracy: 0.8073 - val_loss: 1.1521 - val_accuracy: 0.6551
Epoch 6/10
4/4 [==============================] - 488s 127s/step - loss: 1.1785 - accuracy: 0.6515 - val_loss: 0.8746 - val_accuracy: 0.8293
Epoch 7/10
4/4 [==============================] - 484s 125s/step - loss: 1.0188 - accuracy: 0.8188 - val_loss: 0.8167 - val_accuracy: 0.8223
Epoch 8/10
4/

In [15]:
test_loss, test_acc = model.evaluate(np.array(x_test), np.array(y_te_one_hot), verbose=0)
print(test_acc)
##Evaluating Sensitivity, Accuracy and Kappa scores
y_prob = model.predict(x_test) 
Y_pred = y_prob.argmax(axis=-1)

cm1 = confusion_matrix(y_test,Y_pred)
print("confusion matrix \n",cm1)

0.801047146320343
confusion matrix 
 [[376  87]
 [ 27  83]]


In [16]:
precision = sklearn.metrics.precision_score(y_test,Y_pred,average='micro')
print(precision)

0.8010471204188482


In [17]:
accuracy=np.diag(cm1).sum()/cm1.sum().sum()
print(accuracy)

0.8010471204188482


In [18]:
recall = sklearn.metrics.recall_score(y_test,Y_pred,average='micro')
print(recall)

0.8010471204188482


In [19]:
K_cappa = sklearn.metrics.cohen_kappa_score(y_test,Y_pred)
print(K_cappa)

0.4690994798439533
